In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
import sys, os
pbin_dir = os.path.dirname(sys.executable)
%env PBIN_DIR={pbin_dir}
os.environ['PATH'] = f'{os.environ["PBIN_DIR"]}:{os.environ["PATH"]}'
print(os.environ['PATH'])

env: PBIN_DIR=/share_6/work/solepro-moriya/jupyter102/venv-3.9.10-cikm/bin
/share_6/work/solepro-moriya/jupyter102/venv-3.9.10-cikm/bin:/root/.pyenv/versions/3.9.10/bin:/root/.pyenv/libexec:/root/.pyenv/plugins/python-build/bin:/root/.pyenv/shims:/root/.pyenv/bin:/usr/local/nvidia/bin:/usr/local/cuda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin


In [36]:
#!${PBIN_DIR}/pip install pydash
import pydash
import pickle
vocab, word_embeddings = pickle.load(file=open(f"vocab_and_embeddings_msc.pkl", 'rb'))
import torchtext
from torchtext.data import get_tokenizer
tokenizer = get_tokenizer("basic_english")
unkkey=list(vocab.keys())[-1]
print(f'unkkey: {unkkey}')
unkv = vocab[unkkey]
print(f'unkv: {unkv}')
import fileinput, json, copy, random
from tqdm.auto import tqdm
def conv_uttr(x,seqlen,vocab,unkkey):
    x = tokenizer(x)
    seqlenover=False
    if len(x)>seqlen:
        seqlenover=True
        x=x[:seqlen]
    x = ['<PAD>'] * (seqlen - len(x)) + x
    x = [vocab.get(y, vocab[unkkey]) for y in x]
    return [x,seqlenover]
def randomize_cands(cands,true_index):
    inds = list(range(len(cands)))
    inds=random.sample(inds,len(inds))
    cands=[cands[i] for i in inds]
    label=inds.index(true_index)
    return [cands,label]
def add_response_cands(all_uttrs,cands,count,exclude_responses):
    l=len(cands)+count
    while True:
        if len(cands) == l:
            break
        u = random.choice(all_uttrs)
        if u in cands:
            continue
        if u in exclude_responses:
            continue
        cands.append(u)
    return cands
def get_tsv_groups(div,grouplen):
    with open(f'/share_7/projects/hais/data/msc_bertfp/last_history/{div}.tsv',encoding='utf-8') as f:
        lines=[x.strip().split('\t') for x in f.read().strip().split('\n')]
    groups=[]
    group = None
    for line in lines:
        if line[0] == '1':
            if group is not None:
                assert(len(group)==grouplen)
                groups.append(group)
            group = []
        session,author=line[1].split(' ',maxsplit=1)
        group.append([session,author,line[-1].strip()])
    assert(len(group)==grouplen)
    groups.append(group)
    return groups
def create_data(div,tsvdiv,tsvgrouplen,respcount,seqlen,outfile):
    convs = {}
    prev_count = 0
    settings=[
        [f'/share_6/work/solepro-moriya/jupyter102/selfplay/msc/msc_dialogue/session_2/{div}.txt', True],
        [f'/share_6/work/solepro-moriya/jupyter102/selfplay/msc/msc_dialogue/session_3/{div}.txt', False],
        [f'/share_6/work/solepro-moriya/jupyter102/selfplay/msc/msc_dialogue/session_4/{div}.txt', False]
    ]
    if div != 'train':
        settings.append([f'/share_6/work/solepro-moriya/jupyter102/selfplay/msc/msc_dialogue/session_5/{div}.txt',False])
    all_uttrs=[]
    if True:
        for inp, prev in settings:
            fi = fileinput.FileInput((inp,))
            for line in fi:
                d = json.loads(line)
                ses=d['dialog'][0]['convai2_id']
                if ' ' in ses:
                    assert(False)
                if prev:
                    t = [[x['text'].strip(),None] for x in d['previous_dialogs'][0]['dialog']] 
                    convs[ses] = {
                        'u':t
                    }
                else:
                    if 'i' not in convs[ses]:
                        #index0のidがおかしい場合があるので避けて取る
                        convs[ses]['i']=[x['id'] for x in d['dialog']][2:4]
                        #if convs[ses]['i'][0]=='56152':
                        #    print(inp)
                        #    print(ses)
                        #    print(d['dialog'][0])
                        #    print(convs[ses])
                        convs[ses]['p']=d['personas']
                convs[ses]['u'] += [[x['text'].strip(),x['id']] for x in d['dialog']]
    for ses in convs:
        all_uttrs+=[x[0] for x in convs[ses]['u']]
    all_uttrs=pydash.uniq(all_uttrs)
    tsv_groups=get_tsv_groups(tsvdiv,tsvgrouplen)
    er1=0
    er2=0
    er3=0
    er4=0
    er5=0
    contextlens=[]
    data=[]
    for tg in tsv_groups:
        ses,ca=tg[0][:2]
        if ses not in convs:
            ses = f'{div}:{ses}'
        if ses not in convs:
            er2+=1
            print('er2')
            print(ses)
            continue
        conv=convs[ses]
        if ca not in conv['i']:
            #たまに異常な値がauthor名になっている
            #print(conv['i'])
            #print(ca)
            #print(ses)
            #print(conv)
            er1+=1
            continue
        cind=conv['i'].index(ca)
        rind=1-cind
        ra=conv['i'][rind]
        history=copy.copy(conv['u'])
        if history[-1][1] != ra:
            history=history[:-1]
        if history[-1][1] != ra:
            #たまに異常な値がauthor名になっている
            er1+=1
            continue
        if history[-1][0]!=tg[0][2]:
            #speaker1が2連続で喋ったりする
            #print(ca)
            #print(ra)
            #print(history)
            #print(tg[0])
            #print(history[-1][0])
            #print(tg[0][2])
            er3+=1
            continue
        if history[-2][1] != ca:
            er4+=1
            continue
        #historyを逆順にたどり、authorが交互になっている部分まで取る（ここまでの確認で最低2utteranceはある）
        hind=0
        context=[]
        while True:
            if history[-1-hind][1]!=ra or history[-2-hind][1]!=ca:
                break
            if len(context)>=30:
                break
            context.insert(0,history[-1-hind][0])
            context.insert(0,history[-2-hind][0])
        contextlens.append(len(context))
        response=context[-1]
        context=context[:-1]
        response_cands=[x[2] for x in tg]
        response_cands=add_response_cands(all_uttrs,response_cands,respcount-tsvgrouplen,[])
        response_cands,label=randomize_cands(response_cands,0)
        data.append([None,context,response,response_cands,label])
    print(f'avgconlen:{sum(contextlens)/len(contextlens)}')
    ud=[]
    rd=[]
    yd=[]
    for persona,context,response,response_cands,label in tqdm(data,desc='write_data'):
        convcon=[]
        for c in context:
            cc,seqover=conv_uttr(c,seqlen,vocab,unkkey)
            if seqover:
                er5+=1
            assert(len(cc)==seqlen)
            convcon.append(cc)
        convcon=[*([[0] * seqlen] * (29 - len(convcon))), *convcon]
        assert(len(convcon)==29)
        for i in range(respcount):
            res=response_cands[i]
            yd.append(1 if res==response else 0)
            ud.append(convcon)
            convres,seqover=conv_uttr(res,seqlen,vocab,unkkey)
            if seqover:
                er5+=1
            assert(len(convres)==seqlen)
            rd.append(convres)
        assert(sum(yd[-respcount:])==1)
    print(f'er1:{er1},er2:{er2},er3:{er3},er4:{er4},er5:{er5}')
    assert(len(ud)==len(rd))
    assert(len(ud)==len(yd))
    with open(outfile,'wb') as f:
        pickle.dump([ud,rd,yd],f)
#redditでは50でよかったが、mscはer5が多いので0になるまで増やす（trainはrandom性があるから0にならないかも）
seqlen=200
seqlen=50
create_data(
    'train',
    'train',
    2,
    10,
    seqlen,
    'tm_train_msc_cemb.pkl'
)
create_data(
    'test',
    'test',
    10,
    10,
    seqlen,
    'tm_test_msc_cemb.pkl'
)

unkkey: 41
unkv: 12527
avgconlen:30.0


write_data: 100%|██████████| 7991/7991 [00:06<00:00, 1200.71it/s]


er1:8,er2:0,er3:1,er4:0,er5:26672
avgconlen:30.0


write_data: 100%|██████████| 1002/1002 [00:00<00:00, 1132.29it/s]


er1:0,er2:0,er3:0,er4:0,er5:5758


In [37]:
!cp tm_test_msc_cemb.pkl dataset/reddit2/dev_reddit2.pkl
!cp tm_test_msc_cemb.pkl dataset/reddit2/test_reddit2.pkl
!cp tm_train_msc_cemb.pkl dataset/reddit2/train_reddit2.pkl
!cp vocab_and_embeddings_msc.pkl dataset/reddit2/vocab_and_embeddings.pkl
